## Classifier Model - Tfidf Vectorizer  

In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from distutils.version import LooseVersion as Version
from sklearn import __version__ as sklearn_version

if Version(sklearn_version) < '0.18':
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split

%matplotlib inline

In [5]:
data_file_path = os.path.join("data","Master List of Preferred Term v21 2020_03_19 (1).xlsx")
excel_file_obj = pd.ExcelFile(data_file_path)
df = excel_file_obj.parse("Compiled")

In [6]:
print(f"No of rows in the dataset:{df.shape}")

No of rows in the dataset:(1168, 3)


In [7]:
## Preprocessing (not sure if this is required) - removing punctuation, removing stopwords, conversion to lowercase, stemming (reduce the number of inflectional form of words appearing in the text) 

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import re

stemmer = PorterStemmer()
words = stopwords.words("english")

df['Processed-MedDRA Preferred Term (PT)'] = df['MedDRA Preferred Term (PT)'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\c299342\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [9]:
## Split the data

In [13]:
label = df["Endpoint"]
split_percent = 0.05
X_train, X_test, y_train, y_test = train_test_split(df['Processed-MedDRA Preferred Term (PT)'], label, test_size=split_percent, random_state = 100)

print(X_train.shape, X_test.shape)

(1109,) (59,)


0                                Serious Arrhythmias
1                                Serious Arrhythmias
2                                Serious Arrhythmias
3                                Serious Arrhythmias
4                                Serious Arrhythmias
                            ...                     
1163    Venous/Pulmonary Artery Thromboembolic Event
1164    Venous/Pulmonary Artery Thromboembolic Event
1165    Venous/Pulmonary Artery Thromboembolic Event
1166    Venous/Pulmonary Artery Thromboembolic Event
1167    Venous/Pulmonary Artery Thromboembolic Event
Name: Endpoint, Length: 1168, dtype: object